This notebook presents an application of the [pydiva2D](./pydiva/pydiva2d.py) module to perform a 2D Diva analysis and then generate figures for the different input and output using the [Folium](https://pypi.python.org/pypi/folium) module.

**Note:** since this module accesses various javascript libraries hosted on CDNs, an internet connection is needed to have a working example directly.    
If you're familiar with [leaflet](http://leafletjs.com/), a better solution would be to create JSON files for the different DIVA inputs and outputs, and the visualise with a selected style.

In [ ]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import plugins
import json
import geojson
import subprocess
from pydiva import pydiva2d
from importlib import reload
from mpl_toolkits.basemap import Basemap 

We set up the logger so that only *info* messages are displayed on screen.

In [ ]:
logger = logging.getLogger('diva2D')
logger.setLevel(logging.DEBUG)
pydiva2d.logger.setLevel(logging.DEBUG)

# Prepare input files and directories

In this example the input files are already created and can be found in the *data* directory.

In [ ]:
datadir = '../data/'
datafile = os.path.join(datadir, 'MLD1.dat')
coastfile = os.path.join(datadir, 'coast.cont')
paramfile = os.path.join(datadir, 'param.par')

We also create a directory *html* where we will save the resulting `Leaflet` maps.

In [ ]:
htmldir = './html/'
if os.path.exists(htmldir):
    logger.debug("Directory {0} already exists".format(htmldir))
else:
    logger.debug("Creating directory {0}".format(htmldir))
    os.makedirs(htmldir)

In [ ]:
divadir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1"

# Run Diva
## Generate the mesh

In [ ]:
mesh2d = pydiva2d.Diva2DMesh().make(divadir, coastfile, paramfile)

## Make the analysis

In [ ]:
analysis2d = pydiva2d.Diva2DResults().make(divadir, contourfile=coastfile, paramfile=paramfile, datafile=datafile)

# Create the maps

In this example the plots are created using the [Folium](https://github.com/python-visualization/folium) module.<br>
In each step of the map creation, we will save a html file containg the different elements.

## Initialise the projection

We will center the map on the region covered by the analysis.    
The zoom level (zoom_start) has to be adapted according to the extension of the region.

In [ ]:
divamap = folium.Map(location=[analysis2d.y.mean() , analysis2d.x.mean()], 
                     min_lat=analysis2d.y.min(), max_lat=analysis2d.y.max(), 
                     min_lon=analysis2d.x.min(), max_lon=analysis2d.x.max(),
                     zoom_start=7)
divamap.save(os.path.join(htmldir, 'basemap.html'))

## Data points

We place a circle for each data location.

In [ ]:
Data = pydiva2d.Diva2DData().read_from(datafile)
for lat, lon in zip(Data.y, Data.x):
    #datapoints = folium.CircleMarker([Data.y, Data.x])
    folium.CircleMarker(location=[lat, lon], radius=5, color='green').add_to(divamap)
    #divamap.add_child(datapoints)
divamap.save(os.path.join(htmldir, 'datapoints.html'))

## Contours

The different contours are read and represented as *Polylines*.

In [ ]:
Contour = pydiva2d.Diva2DContours().read_from(coastfile)
for lons, lats in zip(Contour.x, Contour.y):
    # Note that coordinates are defined as (lat, lon)
    coords = zip(lats, lons)
    folium.PolyLine(locations=coords, color='blue', weight=5, opacity=0.75).add_to(divamap)
divamap.save(os.path.join(htmldir, 'contours.html'))

We want to prepare a figure that has no border, axis, title, ...<br>
It is also necessary to specify the projection ([epsg 3857](http://spatialreference.org/ref/sr-org/6864/)) in order to fit with the map background.

## Analysis

The procedure works in two steps:
1. create a figure (png),
2. add the figure as an overlay to the map.

Another solution, not presented here, consists in creating a [geoJSON](http://geojson.org/) file that can be read by leaflet.

### Generation of a figure

The key option to ensure the figure will fit on the map is [`epsg=3857`](https://epsg.io/3857), which states that we use the same projection as in google-maps and [OpenStreetMap](http://openstreetmap.org/).

In [ ]:
resultfigname = 'analysedfield.png'
m = Basemap(llcrnrlon=analysis2d.x.min(), llcrnrlat=analysis2d.y.min(),
            urcrnrlon=analysis2d.x.max(), urcrnrlat=analysis2d.y.max(), 
            resolution = 'l', epsg=3857)
lon, lat = np.meshgrid(analysis2d.x, analysis2d.y)
llon, llat = m(lon, lat)
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
analysis2d.add_to_plot(field='analysis', m=m)
f1 = plt.gca()
f1.axes.get_xaxis().set_ticks([])
f1.axes.get_yaxis().set_ticks([])
# plt.show()
plt.savefig(os.path.join(htmldir, resultfigname), 
            transparent=True, 
            bbox_inches='tight', 
            pad_inches=0)
plt.close()

### Create overlay

Now the figure is created, it can be added as an overlay to the map.      
To do so we use the [ImageOverlay](https://github.com/python-visualization/folium/blob/master/folium/plugins/image_overlay.py) method. Note that one can also use a numpy array as input, but here we prefer to first generate the figure in order to have more control on its aspect.

In [ ]:
folium.plugins.ImageOverlay(os.path.join(htmldir, resultfigname),
                            bounds=[[analysis2d.y.min().tolist(), analysis2d.x.min().tolist()], 
                                    [analysis2d.y.max().tolist(), analysis2d.x.max().tolist()]], 
                            opacity=0.8
                           ).add_to(divamap)
divamap.save(os.path.join(htmldir, 'results.html'))

## Finite-element mesh

We follow the same procedure as for the analysed field:
* generate a figure,
* add it as an overlay.

Again, we could also create a geoJSON file representing the mesh.

In [ ]:
meshfigname = 'mesh.png'
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
mesh2d.add_to_plot(m=m, linewidth=.2, color='k')
f1 = plt.gca()
f1.axes.get_xaxis().set_ticks([])
f1.axes.get_yaxis().set_ticks([])
# plt.show()
plt.savefig(os.path.join(htmldir, meshfigname), dpi=300,
            transparent=True, 
            bbox_inches='tight', 
            pad_inches=0)
plt.close()

In [ ]:
folium.plugins.ImageOverlay(os.path.join(htmldir, meshfigname),
                            bounds=[[analysis2d.y.min().tolist(), analysis2d.x.min().tolist()], 
                                    [analysis2d.y.max().tolist(), analysis2d.x.max().tolist()]]
                           ).add_to(divamap)
divamap.save(os.path.join(htmldir, 'mesh.html'))

# Visualisation

We have create one html file per level of plot: 
* [data points](./html/datapoints.html), 
* [contours](./html/contours.html), 
* [analysed fields](./html/results.html)
* [mesh](./html/mesh.html).

One can easily combine and customise the different elements of the plot to obtained the desired Leaflet map.